<a href="https://colab.research.google.com/github/biancabodo/RealFakeImageClassifier/blob/main/ImageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"biancabodo","key":"e8b8a9deab904a766105f81d6addbafb"}'}

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle datasets download -d ciplab/real-and-fake-face-detection --force

100% 431M/431M [00:06<00:00, 114MB/s] 
100% 431M/431M [00:06<00:00, 73.8MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

In [ ]:
%mv real_and_fake_face/ input_data
%ls

gdrive/  input_data/  kaggle.json  real_and_fake_face_detection/  sample_data/


In [ ]:
import pathlib

data_dir = pathlib.Path('/content/input_data/')
training_fake = pathlib.Path('/content/input_data/training_fake')
training_real = pathlib.Path('/content/input_data/training_real')

In [ ]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 15.3MB/s 


In [ ]:
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN
from PIL import Image
# load image from folder
filename = '/content/input_data/training_fake/mid_137_1011.jpg'

def draw_image_with_boxes(filename, result_list):
    # load the image
    data = pyplot.imread(filename)
    # plot the image
    pyplot.imshow(data)
    # get the context for drawing boxes
    ax = pyplot.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        # draw the box
        ax.add_patch(rect)
    # show the plot
    pyplot.show()
    return(x,y,x+width,y+height)
    
detector = MTCNN()
# detect faces in the image
pixels = pyplot.imread(filename)
print(pixels)
faces = detector.detect_faces(pixels)
# display faces on the original image
draw_image_with_boxes(filename, faces)

In [ ]:
%mkdir data
%cd data/
%mkdir fake
%mkdir real


/content/data


In [ ]:
%cd ..

/content


In [ ]:
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN
from PIL import Image
import os 

## change the folder name accordingly for training and testing
path = '/content/input_data/'

folders = os.listdir(path)
print(folders)
#folders = folders[1:] ## [1:] to remove .ds_store folder if it is made automatically otherwise just use folder

## Iterate over the folder and detect and crop faces and save them in respective folder
for subs in folders:
    for files in os.listdir(path+subs):
        try:
            if 'fake' in path+subs+files and 'jpg' in path+subs+files:
                print(path+subs+'/'+files)
                pixels = pyplot.imread(path+subs+'/'+files)
                faces = detector.detect_faces(pixels)
                x, y, width, height = faces[0]['box']
                #coordinates = tuple(faces[0]['box'])
                Image.fromarray(pixels).crop((x, y, x + width, y + height)).save('/content/data/fake/'+files)
            elif 'real' in path+subs+files and 'jpg' in path+subs+files:
                print(path+subs+'/'+files)
                pixels = pyplot.imread(path+subs+'/'+files)
                faces = detector.detect_faces(pixels)
                x, y, width, height = faces[0]['box']
                #coordinates = tuple(faces[0]['box'])
                Image.fromarray(pixels).crop((x, y, x + width, y + height)).save('/content/data/real/'+files)
        except (IndexError or SystemError):
            print('Face Not Found')

In [ ]:

from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:

model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (200,200,3),activation = 'relu',padding='same',))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255.0,
                             validation_split = 0.2
                             
                             )

train = datagen.flow_from_directory('/content/data/',
                                    classes={
                                        'fake':1,
                                        'real':0
                                    },
                                    class_mode='binary',
                                    batch_size=64,
                                    target_size=(200,200),
                                    subset = 'training')

val = datagen.flow_from_directory('/content/data/',
                                  classes={
                                        'fake':1,
                                        'real':0
                                    },
                                    class_mode='binary',
                                    batch_size=64,
                                    target_size=(200,200),
                                    subset = 'validation')
print(val)

history = model.fit_generator(train,
                              validation_data=(val),
                              epochs = 50,
                              steps_per_epoch=len(train),
                              validation_steps=val.samples
                              )


In [ ]:
models_path = '/content/gdrive/MyDrive/uclais/dataset/dataset/data/dataset/dataset/test/models/'
model_json = model.to_json()
with open(models_path+'model.json','w') as json_file:
    json_file.write(model_json)

model.save_weights(models_path+'model.h5')

In [ ]:
%mkdir /content/final_test/
%ls

mkdir: cannot create directory ‘/content/final_test/’: File exists
model.h5  model.json


In [ ]:
%cd /content/final_test/
%rm *.jpg
%ls

/content/final_test


In [ ]:

from keras.models import load_model, model_from_json
from PIL import Image
import json
import numpy as np 
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import pandas as pd
import shutil

folder_path = '/content/gdrive/MyDrive/uclais/dataset/dataset/data/dataset/dataset/test/test/'
cropped_faces_test_path = '/content/final_test/'
model_json_path = '/content/gdrive/MyDrive/uclais/dataset/dataset/data/dataset/dataset/test/models/model.json'
model_weights_path = '/content/gdrive/MyDrive/uclais/dataset/dataset/data/dataset/dataset/test/models/model.h5'

json_file = open(model_json_path,'r')
loaded_model = json_file.read()
json_file.close()

model = model_from_json(loaded_model)
model.load_weights(model_weights_path)

img_width, img_height = 200,200

# Load Image 
images = []
folder_images = os.listdir(folder_path)

for img in folder_images:
  try:
      print(img)
      pixels = pyplot.imread(folder_path + img)
      faces = detector.detect_faces(pixels)
      x, y, width, height = faces[0]['box']
      Image.fromarray(pixels).crop((x, y, x + width, y + height)).save(cropped_faces_test_path+img)
  except(IndexError or SystemError):
      print('bagi pula in ' + img)
      shutil.move(folder_path + img, cropped_faces_test_path)


cropped_img_list = os.listdir(cropped_faces_test_path)
cropped_img_list.sort(key=lambda file_name: int(file_name.split('.')[0]))
print(cropped_img_list)
for img in cropped_img_list:
    img = os.path.join(cropped_faces_test_path, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    # im = np.asarray(im)
    #img = np.reshape(img,(1,img.shape[0],img.shape[1],img.shape[2]))
    images.append(img)


# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)
print(classes)
for i,_ in enumerate(classes):
  classes[i] = 1 - classes[i]

print(classes)

result_df = pd.DataFrame()
result_df["filename"] = list(cropped_img_list)
result_df["label"] = classes

result_df.head()
result_df.to_excel(models_path+'subms2.xlsx') 


In [ ]:
!git init

Initialized empty Git repository in /content/.git/


In [ ]:
!git 

In [ ]:
!git config — global user.email “bodo.bianca@yahoo.com”
!git config — global user.name “biancabodo”

usage: git config [<options>]

Config file location
    --global              use global config file
    --system              use system config file
    --local               use repository config file
    -f, --file <file>     use given config file
    --blob <blob-id>      read config from given blob object

Action
    --get                 get value: name [value-regex]
    --get-all             get all values: key [value-regex]
    --get-regexp          get values for regexp: name-regex [value-regex]
    --get-urlmatch        get value specific for the URL: section[.var] URL
    --replace-all         replace all matching variables: name value [value_regex]
    --add                 add a new variable: name value
    --unset               remove a variable: name [value-regex]
    --unset-all           remove all matches: name [value-regex]
    --rename-section      rename section: old-name new-name
    --remove-section      remove a section: name
    -l, --list            list all
 

In [ ]:
!git add -A

In [ ]:
!git commit -m “ImageClassifier”


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@fa2afb1af462.(none)')


In [ ]:
!git remote add origin https://<biancabodo>:<hovhuT-wagnaw-zyzku1>@github.com/<biancabodo>/ImageClassifier

/bin/bash: biancabodo: No such file or directory
